# Model Building with BoFire

This notebooks shows how to setup and analyze models trained with BoFire. It is stil WIP.

## Imports

In [6]:
from bofire.domain.features import (
    ContinuousInput,
    ContinuousOutput,
    InputFeatures,
    OutputFeatures,
)
from bofire.models.torch_models import SingleTaskGPModel
from bofire.utils.enum import RegressionMetricsEnum

## Problem Setup

For didactic purposes, we sample data from a Himmelblau benchmark function and use them to train a SingleTaskGP.

In [7]:
# Todo: replace this after JDs PR is ready.
input_features = InputFeatures(
        features=[
            ContinuousInput(key=f"x_{i+1}", lower_bound=-4, upper_bound=4)
            for i in range(2)
        ]
    )
output_features = OutputFeatures(features=[ContinuousOutput(key="y")])
experiments = input_features.sample(n=50)
experiments.eval("y=((x_1**2 + x_2 - 11)**2+(x_1 + x_2**2 -7)**2)", inplace=True)
experiments["valid_y"] = 1

## Cross Validation
### Run the cross validation

In [8]:
model = SingleTaskGPModel(
    input_features=input_features,
    output_features=output_features,
)
train_cv, test_cv = model.cross_validate(experiments, folds=5)

### Analyze the cross validation

Plots are added in a future PR.

In [4]:
# Performance on test sets
test_cv.get_metrics(combine_folds=True)

,MAE,MSD,R2,MAPE,PEARSON,SPEARMAN,FISHER
0,7.0017,147.061554,0.943717,0.156511,0.97159,0.97916,4.952116e-12


In [5]:
display(test_cv.get_metrics(combine_folds=False))
display(test_cv.get_metrics(combine_folds=False).describe())

,MAE,MSD,R2,MAPE,PEARSON,SPEARMAN,FISHER
0,10.339322,364.933979,0.892727,0.118647,0.972169,1.000000,0.003968
1,4.478753,34.141282,0.990493,0.299760,0.995986,1.000000,0.003968
2,6.422695,93.220180,0.942858,0.070685,0.976744,0.866667,0.103175
3,3.264905,38.618496,0.974880,0.094516,0.989166,0.963636,0.003968
4,10.502826,204.393833,0.902698,0.198947,0.978679,0.963636,0.003968


,MAE,MSD,R2,MAPE,PEARSON,SPEARMAN,FISHER
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,7.001700,147.061554,0.940731,0.156511,0.982549,0.958788,0.023810
std,3.318951,139.790166,0.043005,0.093494,0.009756,0.054613,0.044366
min,3.264905,34.141282,0.892727,0.070685,0.972169,0.866667,0.003968
25%,4.478753,38.618496,0.902698,0.094516,0.976744,0.963636,0.003968
50%,6.422695,93.220180,0.942858,0.118647,0.978679,0.963636,0.003968
75%,10.339322,204.393833,0.974880,0.198947,0.989166,1.000000,0.003968
max,10.502826,364.933979,0.990493,0.299760,0.995986,1.000000,0.103175
